In [ ]:
pip install pyLDAvis

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import spacy
import gensim
from gensim import corpora
 
# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
file1 = open('/content/data-2660.txt','r')
s = file1.read()
documents = s.split(",")

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
       return output


In [ ]:
tokenized_documents = lemmatization(documents)

In [ ]:
dictionary = corpora.Dictionary(tokenized_documents)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_documents]

In [ ]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel
 
# Build LDA modeoml
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=5, chunksize=5)

In [ ]:
lda_model.print_topics()

[(0,
  '0.012*"people" + 0.009*"many" + 0.008*"way" + 0.008*"right" + 0.008*"person" + 0.008*"real" + 0.007*"possible" + 0.007*"thing" + 0.007*"year" + 0.007*"phone"'),
 (1,
  '0.044*"page" + 0.016*"search" + 0.016*"tool" + 0.016*"site" + 0.015*"category" + 0.015*"name" + 0.014*"file" + 0.013*"user" + 0.012*"action" + 0.012*"good"'),
 (2,
  '0.017*"card" + 0.013*"web" + 0.012*"use" + 0.012*"-" + 0.012*"key" + 0.011*"server" + 0.010*"service" + 0.010*"password" + 0.009*"network" + 0.009*"email"'),
 (3,
  '0.023*"product" + 0.019*"money" + 0.019*"cart" + 0.018*"wishlist" + 0.013*"bank" + 0.012*"order" + 0.010*"process" + 0.010*"transaction" + 0.010*"office" + 0.010*"hour"'),
 (4,
  '0.116*"video" + 0.093*"porn" + 0.039*"talk" + 0.032*"child" + 0.017*"log" + 0.016*"porno" + 0.015*"day" + 0.013*"photo" + 0.011*"user" + 0.011*"hist"')]

In [ ]:
import pyLDAvis
import pyLDAvis.gensim 
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.150339 -0.273602       1        1  33.463637
0      0.197294  0.050657       2        1  24.644696
2      0.172601 -0.117182       3        1  20.281446
3      0.045301  0.190816       4        1  11.182475
4     -0.264856  0.149311       5        1  10.427747, topic_info=       Term         Freq        Total Category  logprob  loglift
1933  video  3876.000000  3876.000000  Default  30.0000  30.0000
1926   porn  3038.000000  3038.000000  Default  29.0000  29.0000
473    page  4840.000000  4840.000000  Default  28.0000  28.0000
576    talk  1719.000000  1719.000000  Default  27.0000  27.0000
461   child  1111.000000  1111.000000  Default  26.0000  26.0000
...     ...          ...          ...      ...      ...      ...
180    last   281.194588   683.106915   Topic5  -4.7468   1.3731
443    user   362.964373  1879.695699   Topic5  -4.4915   0.6161
473    page   307.265910  4840.146473   Topic5  -4.6581  -0.4963
1009    tag   119.788793   286.803467   Topic5  -5.6001   1.3876
193     new   138.411768   949.868838   Topic5  -5.4556   0.3346

[299 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
451       3  0.997045      #
451       5  0.002849      #
271       2  0.179262      %
271       3  0.819902      %
501       1  0.194289      -
...     ...       ...    ...
450       2  0.865651   year
450       3  0.024866   year
450       5  0.001554   year
1907      2  0.021561  young
1907      5  0.975628  young

[485 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3, 4, 5])

In [ ]:
from gensim import models
#lda_model.save('lda.model')
lda_model =  models.LdaModel.load('lda.model')

In [ ]:
def getMax(probabilities):
  maxProb = 0
  ans = 0
  #print(probabilities)
  for i in range(len(probabilities)):
    if probabilities[i][1] > maxProb:
      maxProb = probabilities[i][1]
      ans = probabilities[i][0]
  return ans

In [ ]:
labels = []
i = 0
for i in range(len(tokenized_documents)):
  labels.append(getMax(lda_model.get_document_topics(dictionary.doc2bow(tokenized_documents[i])))) 
  i = i+1


i = 0
for label in labels:
  if label == 1:
    labels[i] = "wiki"
  elif label == 0:
    labels[i] = "unk"
  elif label == 2:
    labels[i] = "software"
  elif label == 3:
    labels[i] = "marketplace"
  else:
    labels[i] = "porn"
  i = i+1


In [ ]:
with open('finalResult.txt','w+') as writeFile:
  for i in range(len(labels)):
    writeFile.write(str(i)+","+labels[i]+"\n")


In [ ]:
gow=["wiki","software","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","marketplace","software","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","porn","wiki","wiki","wiki","wiki","porn","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","software","unk","wiki","software","wiki","wiki","wiki","software","unk","unk","wiki","wiki","marketplace","wiki","software","wiki","marketplace","wiki","marketplace","wiki","wiki","unk","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","porn","wiki","wiki","marketplace","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","software","wiki","wiki","software","wiki","wiki","wiki","unk","wiki","wiki","unk","wiki","software","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","software","unk","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","marketplace","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","unk","unk","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","marketplace","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","software","wiki","marketplace","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","porn","unk","wiki","wiki","software","wiki","unk","wiki","wiki","marketplace","wiki","wiki","wiki","marketplace","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","software","software","wiki","software","wiki","software","software","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","wiki","software","unk","wiki","wiki","wiki","software","wiki","wiki","software","wiki","unk","wiki","unk","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","unk","wiki","wiki","unk","wiki","wiki","marketplace","unk","unk","unk","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","software","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","software","wiki","wiki","wiki","marketplace","software","unk","wiki","wiki","unk","unk","marketplace","marketplace","marketplace","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","unk","wiki","marketplace","wiki","wiki","wiki","software","wiki","wiki","wiki","marketplace","software","wiki","wiki","wiki","unk","unk","wiki","unk","wiki","unk","wiki","wiki","unk","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","software","wiki","software","software","marketplace","unk","wiki","marketplace","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","software","wiki","wiki","software","wiki","wiki","unk","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","unk","wiki","wiki","wiki","marketplace","wiki","unk","wiki","wiki","software","wiki","wiki","unk","unk","wiki","wiki","wiki","wiki","marketplace","software","marketplace","software","unk","unk","unk","unk","wiki","wiki","wiki","unk","wiki","wiki","porn","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","marketplace","wiki","wiki","unk","marketplace","marketplace","wiki","wiki","wiki","wiki","unk","unk","porn","unk","wiki","wiki","wiki","software","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","marketplace","wiki","wiki","wiki","wiki","wiki","software","unk","wiki","unk","marketplace","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","marketplace","wiki","unk","marketplace","wiki","wiki","unk","wiki","wiki","unk","software","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","unk","unk","porn","wiki","wiki","porn","wiki","wiki","software","wiki","wiki","porn","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","software","unk","unk","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","software","software","software","wiki","wiki","unk","wiki","wiki","wiki","marketplace","software","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","software","unk","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","wiki","software","porn","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","software","wiki","wiki","wiki","software","wiki","wiki","software","wiki","wiki","wiki","software","wiki","wiki","wiki","unk","marketplace","wiki","wiki","porn","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","unk","wiki","wiki","unk","wiki","unk","wiki","unk","wiki","wiki","wiki","wiki","unk","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","unk","wiki","software","wiki","marketplace","wiki","unk","wiki","wiki","unk","unk","porn","wiki","software","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","unk","wiki","marketplace","wiki","unk","wiki","wiki","wiki","wiki","marketplace","wiki","unk","wiki","wiki","marketplace","marketplace","unk","software","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","marketplace","software","wiki","wiki","wiki","wiki","wiki","unk","software","wiki","wiki","wiki","porn","wiki","wiki","software","wiki","unk","wiki","wiki","software","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","software","unk","wiki","unk","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","unk","wiki","wiki","software","wiki","unk","software","wiki","unk","wiki","wiki","wiki","wiki","wiki","software","wiki","software","wiki","wiki","wiki","wiki","software","wiki","wiki","unk","wiki","marketplace","marketplace","porn","marketplace","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","software","wiki","wiki","marketplace","wiki","unk","wiki","software","software","software","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","software","wiki","wiki","wiki","porn","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","marketplace","wiki","wiki","unk","wiki","wiki","wiki","unk","wiki","wiki","marketplace","unk","wiki","wiki","software","wiki","software","wiki","software","marketplace","wiki","wiki","unk","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","software","wiki","software","wiki","software","wiki","wiki","unk","wiki","marketplace","wiki","software","wiki","software","wiki","software","wiki","software","unk","wiki","unk","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","software","software","wiki","wiki","wiki","unk","software","wiki","software","software","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","unk","marketplace","marketplace","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","software","software","software","software","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","unk","wiki","unk","software","software","software","wiki","wiki","wiki","software","software","software","wiki","software","wiki","software","wiki","marketplace","wiki","software","wiki","wiki","unk","unk","unk","marketplace","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","software","unk","wiki","marketplace","porn","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","marketplace","wiki","unk","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","marketplace","software","wiki","software","software","wiki","unk","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","porn","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","software","marketplace","wiki","marketplace","software","software","wiki","unk","wiki","wiki","wiki","unk","wiki","wiki","software","wiki","wiki","unk","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","software","software","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","software","wiki","marketplace","wiki","marketplace","marketplace","marketplace","unk","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","marketplace","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","software","porn","wiki","wiki","unk","wiki","marketplace","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","marketplace","software","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","unk","software","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","marketplace","software","wiki","wiki","marketplace","wiki","software","software","software","marketplace","wiki","wiki","wiki","wiki","wiki","software","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","marketplace","wiki","wiki","wiki","wiki","software","marketplace","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","software","wiki","wiki","wiki","wiki","marketplace","software","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","unk","software","marketplace","wiki","software","wiki","marketplace","wiki","software","marketplace","wiki","wiki","wiki","wiki","software","wiki","marketplace","wiki","software","wiki","software","wiki","wiki","software","unk","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","marketplace","software","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","software","wiki","wiki","software","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","software","software","wiki","wiki","wiki","marketplace","wiki","marketplace","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","marketplace","marketplace","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","unk","software","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","marketplace","software","wiki","marketplace","software","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","unk","software","wiki","marketplace","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","software","marketplace","software","wiki","software","marketplace","wiki","wiki","marketplace","porn","wiki","wiki","software","marketplace","marketplace","wiki","unk","marketplace","software","wiki","wiki","wiki","software","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","software","wiki","marketplace","wiki","unk","wiki","wiki","software","software","software","wiki","software","wiki","wiki","wiki","wiki","software","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","software","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","marketplace","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","marketplace","wiki","wiki","software","unk","wiki","marketplace","wiki","software","software","software","software","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","marketplace","marketplace","wiki","wiki","marketplace","wiki","software","wiki","marketplace","wiki","marketplace","wiki","wiki","wiki","unk","porn","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","marketplace","wiki","marketplace","software","wiki","wiki","marketplace","software","wiki","wiki","wiki","software","software","wiki","software","software","wiki","marketplace","wiki","wiki","software","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","software","wiki","marketplace","wiki","software","wiki","wiki","wiki","marketplace","software","unk","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","marketplace","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","software","marketplace","marketplace","marketplace","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","software","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","marketplace","software","wiki","software","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","marketplace","marketplace","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","marketplace","porn","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","software","marketplace","wiki","wiki","wiki","unk","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","unk","wiki","wiki","wiki","wiki","wiki","marketplace","software","software","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","unk","wiki","wiki","unk","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","unk","unk","marketplace","marketplace","unk","marketplace","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","porn","wiki","wiki","wiki","wiki","software","unk","marketplace","wiki","wiki","wiki","wiki","unk","software","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","wiki","software","software","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","unk","unk","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","software","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","software","unk","wiki","software","software","unk","wiki","wiki","unk","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","software","unk","wiki","software","wiki","wiki","software","unk","wiki","marketplace","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","marketplace","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","porn","wiki","marketplace","unk","wiki","wiki","wiki","unk","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","unk","software","wiki","wiki","wiki","wiki","wiki","porn","wiki","wiki","marketplace","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","unk","software","unk","wiki","unk","marketplace","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","unk","software","unk","wiki","wiki","unk","wiki","unk","unk","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","unk","wiki","unk","unk","wiki","wiki","software","unk","porn","wiki","unk","wiki","wiki","unk","unk","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","unk","unk","unk","marketplace","wiki","wiki","wiki","wiki","unk","wiki","wiki","unk","wiki","wiki","wiki","wiki","marketplace","wiki","software","unk","wiki","unk","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","software","unk","wiki","unk","wiki","unk","software","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","marketplace","wiki","software","wiki","wiki","unk","wiki","wiki","unk","unk","wiki","unk","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","software","wiki","wiki","marketplace","wiki","wiki","wiki","unk","wiki","marketplace","wiki","unk","wiki","unk","unk","wiki","wiki","unk","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","unk","wiki","unk","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","software","unk","wiki","wiki","marketplace","wiki","marketplace","wiki","wiki","unk","software","wiki","unk","wiki","unk","wiki","wiki","wiki","marketplace","wiki","unk","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","porn","wiki","unk","wiki","unk","software","wiki","wiki","wiki","unk","wiki","marketplace","wiki","software","wiki","unk","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","unk","wiki","unk","unk","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","marketplace","unk","wiki","wiki","marketplace","wiki","wiki","unk","wiki","software","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","unk","software","unk","wiki","software","wiki","software","unk","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","unk","unk","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","unk","wiki","wiki","wiki","software","unk","wiki","software","wiki","wiki","wiki","wiki","marketplace","wiki","porn","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","unk","wiki","unk","wiki","software","wiki","unk","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","unk","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","unk","wiki","unk","wiki","wiki","wiki","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","software","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","unk","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","unk","wiki","wiki","marketplace","unk","unk","wiki","wiki","wiki","wiki","wiki","unk","unk","wiki","unk","unk","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","unk","wiki","wiki","unk","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","wiki","wiki","marketplace","marketplace","unk","wiki","wiki","wiki","unk","wiki","wiki","unk","unk","wiki","wiki","wiki","wiki","wiki","software","unk","marketplace","wiki","wiki","wiki","wiki","wiki","software","wiki","unk","wiki","wiki","wiki","wiki","software","unk","wiki","marketplace","wiki","marketplace","software","wiki","wiki","wiki","marketplace","wiki","wiki","wiki","software"i]

In [ ]:
counter = 0
for i in range(len(labels)):
  if labels[i] == gow[i]:
    counter = counter + 1
accuracy = (counter/len(documents)*100)
print('Accuracy: '+ str(accuracy) + ' %')

Accuracy: 78.04511278195488 %
